### Workbook 5 - Recognising birds from FreeField1010 recordings using a Convolutional Neural Network 

The previous 4 workbooks have been processing audio the UrbanSound8K data set, this was chosen because it provided a large and varied set of samples, belonging to just 10 classes, which made it an ideal basis for developing and testing a general audio classifier. 

As the goal of this project is to recognise birdsong, the next logical step is to try it with actual recordings of birds. Unfortunately it seems difficult to find an equivalent of the UrbanSound8k data for birds, i.e. one where the recordings are labelled with the bird species, but I did find some interesting data sets with binary labels at http://machine-listening.eecs.qmul.ac.uk/bird-audio-detection-challenge/

The binary labels are indicate that a human listener has stated that either they believe a bird is present, or that no birds are present. No species information is included, so the goal here is bird detection rather than identification, i.e. to find segments of 24-hour automated field recordings that might have interesting content, which can be forwarded to human experts who will identify what might be present.

So here I'm going to adapt the CNN produced in workbook 4, and train it on the FreeField1010 data set.


In [2]:
import glob
import os
import librosa
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

### Feature Extraction

This is the feature extraction code, it uses a similar process to that described in workbook 4. You'll only need to run this once, to obtain the feature data from the raw recordings.

In [2]:
def windows(data, window_size):
    start = 0
    while start < len(data):
        yield start, start + window_size
        start += (window_size / 2)

def extract_features(parent_dir,sub_dirs,file_ext="*.wav",bands = 60, frames = 41):
    window_size = 512 * (frames - 1)
    log_specgrams = []
    labels = []
    for l, sub_dir in enumerate(sub_dirs):
        for fn in glob.glob(os.path.join(parent_dir, sub_dir, file_ext)):
            sound_clip,s = librosa.load(fn)
            
            idstr = os.path.basename(fn).split('.wav')[0]
            if idstr not in my_dict.keys():
                print "No such entry ", idstr, 'skipping'
                continue
                
            label = my_dict[idstr]    
            
            for (start,end) in windows(sound_clip,window_size):
                if(len(sound_clip[start:end]) == window_size):
                    signal = sound_clip[start:end]
                    melspec = librosa.feature.melspectrogram(signal, n_mels = bands)
                    logspec = librosa.logamplitude(melspec)
                    logspec = logspec.T.flatten()[:, np.newaxis].T
                    log_specgrams.append(logspec)
                    labels.append(label)
            
    log_specgrams = np.asarray(log_specgrams).reshape(len(log_specgrams),bands,frames,1)
    features = np.concatenate((log_specgrams, np.zeros(np.shape(log_specgrams))), axis = 3)
    for i in range(len(features)):
        features[i, :, :, 1] = librosa.feature.delta(features[i, :, :, 0])
    
    return np.array(features), np.array(labels,dtype = np.int)

def one_hot_encode(labels):
    n_labels = len(labels)
    n_unique_labels = len(np.unique(labels))
    one_hot_encode = np.zeros((n_labels,n_unique_labels))
    one_hot_encode[np.arange(n_labels), labels] = 1
    return one_hot_encode

The following code extracts features from the raw FreeField1010 recordings.

In [8]:
# create a dictionary from the labels file
from numpy import loadtxt
data_dir = "data/ffbird-np-cnn"
filename = data_dir + "/ff1010bird_metadata.csv"
lines = np.genfromtxt(filename, delimiter=",", skip_header=1, dtype=None)
my_dict = dict()
for i in range(len(lines)):
    my_dict[str(lines[i][0])] = lines[i][1]
    
print "Entries:", len(my_dict) 

def assure_path_exists(path):
    mydir = os.path.join(os.getcwd(), path)
    if not os.path.exists(mydir):
        os.makedirs(mydir)
        
# use this to process the audio files into numpy arrays
def save_folds(data_dir):
    for k in range(1,9):
        fold_name = 'fold' + str(k)
        print "\nSaving " + fold_name
        features, labels = extract_features(parent_dir, [fold_name])
        labels = one_hot_encode(labels)
        
        print "Features of", fold_name , " = ", features.shape
        print "Labels of", fold_name , " = ", labels.shape
        
        feature_file = os.path.join(data_dir, fold_name + '_x.npy')
        labels_file = os.path.join(data_dir, fold_name + '_y.npy')
        np.save(feature_file, features)
        print "Saved " + feature_file
        np.save(labels_file, labels)
        print "Saved " + labels_file

# update and uncomment the following lines if you want to run feature extraction for yourself   
#parent_dir = "path-to-the-freefield1010-data-set"
#assure_path_exists(data_dir)
#save_folds(data_dir)

Entries: 7690


### Reload Extracted Features

In [3]:
import scipy
# this is used to load the folds incrementally
data_dir = "data/ffbird-np-cnn/np"

def load_folds(folds):
    subsequent_fold = False
    for k in range(len(folds)):
        fold_name = 'fold' + str(folds[k])
        feature_file = os.path.join(data_dir, fold_name + '_x.npy')
        labels_file = os.path.join(data_dir, fold_name + '_y.npy')
        loaded_features = np.load(feature_file)
        loaded_labels = np.load(labels_file)
        print fold_name, "features: ", loaded_features.shape

        if subsequent_fold:
            features = np.concatenate((features, loaded_features))
            labels = np.concatenate((labels, loaded_labels))
        else:
            features = loaded_features
            labels = loaded_labels
            subsequent_fold = True
        
    return features, labels

train_x, train_y = load_folds([1,2])
print "train_x: ", train_x.shape
print "2 train_y: ", train_y.shape

train_y = scipy.delete(train_y, 1, 1) 
print "1 train_y: ", train_y.shape

valid_x, valid_y = load_folds([8])
valid_y = scipy.delete(valid_y, 1, 1) 
print "valid_x: ", valid_x.shape
print "valid_y: ", valid_y.shape

test_x, test_y = load_folds([7])
test_y = scipy.delete(test_y, 1, 1) 
print "test_x:", test_x.shape
print "test_y:", test_y.shape

fold1 features:  (19400, 60, 41, 2)
fold2 features:  (19400, 60, 41, 2)
train_x:  (38800, 60, 41, 2)
2 train_y:  (38800, 2)
1 train_y:  (38800, 1)
fold8 features:  (18000, 60, 41, 2)
valid_x:  (18000, 60, 41, 2)
valid_y:  (18000, 1)
fold7 features:  (19400, 60, 41, 2)
test_x: (19400, 60, 41, 2)
test_y: (19400, 1)


### Training a Convolutional Neural Network with Keras and TensorFlow

In [4]:
frames = 41
bands = 60

feature_size = bands * frames #60x41
num_labels = test_y.shape[1]
num_channels = 2

print "Labels:", num_labels
print "Feature size:", feature_size

Labels: 1
Feature size: 2460


In [5]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adam, Adagrad, SGD
from keras.callbacks import EarlyStopping
from keras.regularizers import l2, activity_l2
from keras.utils import np_utils

np.random.seed(0)

Using TensorFlow backend.


In [6]:
from sklearn.metrics import roc_auc_score

def evaluate(model):
    y_prob = model.predict_proba(test_x, verbose=0)
    y_pred = np_utils.probas_to_classes(y_prob)
    y_true = np.argmax(test_y, 1)

    roc = roc_auc_score(test_y, y_prob)
    print "ROC:",  round(roc,3)

    # evaluate the model
    score, accuracy = model.evaluate(test_x, test_y, batch_size=32)
    print("Accuracy = {:.2f}".format(accuracy))
    
    return roc, accuracy

In [7]:
# implements CNN described in https://arxiv.org/pdf/1608.04363.pdf

def build_model():
    
    model = Sequential()
    # input: 60x41 data frames with 2 channels => (60,41,2) tensors

    # filters of size 3x3 - paper describes using 5x5, but their input data is 128x128
    f_size = 3

    # Layer 1 - 24 filters with a receptive field of (f,f), i.e. W has the
    # shape (24,1,f,f).  This is followed by (4,2) max-pooling over the last
    # two dimensions and a ReLU activation function
    model.add(Convolution2D(24, f_size, f_size, border_mode='same', input_shape=(bands, frames, num_channels)))
    model.add(MaxPooling2D(pool_size=(4, 2)))
    model.add(Activation('relu'))

    # Layer 2 - 48 filters with a receptive field of (f,f), i.e. W has the 
    # shape (48, 24, f, f). Like L1 this is followed by (4,2) max-pooling 
    # and a ReLU activation function.
    model.add(Convolution2D(48, f_size, f_size, border_mode='same'))
    model.add(MaxPooling2D(pool_size=(4, 2)))
    model.add(Activation('relu'))

    # Layer 3 - 48 filters with a receptive field of (f,f), i.e. W has the
    # shape (48, 48, f, f). This is followed by a ReLU but no pooling.
    model.add(Convolution2D(48, f_size, f_size, border_mode='valid'))
    model.add(Activation('relu'))

    # flatten output into a single dimension, let Keras do shape inference
    model.add(Flatten())

    # Layer 4 - a fully connected NN layer of 64 hidden units, L2 penalty of 0.001
    model.add(Dense(64, W_regularizer=l2(0.001)))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    # Layer 5 - an output layer with one output unit per class, with L2 penalty, 
    # followed by a softmax activation function
    model.add(Dense(num_labels, W_regularizer=l2(0.001)))
    model.add(Dropout(0.5))
    # softmax on single output will always normalise the value to 1.0, so need sigmoid
    model.add(Activation('sigmoid'))

    # create an optimiser
    # adagrad = Adagrad(lr=0.01, epsilon=1e-08, decay=0.0)
    sgd = SGD(lr=0.001, momentum=0.9, decay=0.0, nesterov=True)

    # compile and fit model, reduce epochs if you want a result faster
    # the validation set is used to identify parameter settings (epoch) that achieves 
    # the highest classification accuracy (note binary rather than categorial crossentropy)
    model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=sgd)
    
    return model

In [8]:
# the ModelCheckpoint callback automatically saves the current model's weights to a file whenever the validation accuracy improves
earlystop = EarlyStopping(monitor='val_loss', patience=1, verbose=0, mode='auto')

print("Building model...")
model = build_model()

# now fit the model to the training data, evaluating loss against the validation data
print("Training model...")
model.fit(train_x, train_y, validation_data=(valid_x, valid_y), callbacks=[earlystop], batch_size=128, nb_epoch=15)
        
# now evaluate the trained model against the unseen test data
print("Evaluating model...")
roc, acc = evaluate(model)
       
print 'R.O.C:', round(roc, 3)
print 'Accuracy:', round(acc, 3)

# best ROC (2 folds FF) = 0.69, acc = 0.77
# best ROC (all 6 folds FF set only) = 0.73
# best ROC (2 sets) = 0.869, acc=0.79 (Adagrad batch=128, patience=2) 50 epochs
# best ROC (2 sets) = 0.87 (SGD batch=128, nesterov, momentum=0.9, lr-0.001) 50 epochs

Building model...
Training model...
Train on 38800 samples, validate on 18000 samples
Epoch 1/15
38800/38800 [==============================] - 158s - loss: 0.7319 - acc: 0.7634 - val_loss: 0.5546 - val_acc: 0.7633
Epoch 2/15
38800/38800 [==============================] - 156s - loss: 0.7204 - acc: 0.7646 - val_loss: 0.6075 - val_acc: 0.7633
Epoch 3/15
38800/38800 [==============================] - 151s - loss: 0.7144 - acc: 0.7646 - val_loss: 0.5822 - val_acc: 0.7665
Evaluating model...
ROC: 0.695
19400/19400 [==============================] - 26s    
Accuracy = 0.77
R.O.C: 0.695
Accuracy: 0.768


### Getting even better results

There is another set of field recordings available from http://machine-listening.eecs.qmul.ac.uk/bird-audio-detection-challenge/ - the Warblr data set. By combining both the FreeField1010 and Warblr data sets I was able to obtain even better results, an F-score accuracy of 0.87, this however took considerable processing on a powerful cloud computing instance.